In [1]:
# this automatically reloads the libraries so you can update them dynamically
%load_ext autoreload
%autoreload 2

from models.clip import CLIPModel
from utils.dataset import make_train_valid_dfs, build_loaders, prepare_data
from utils import config
from utils.train import train_epoch, valid_epoch

import torch
from transformers import DistilBertTokenizer
import tqdm.auto as tqdm

In [3]:
prepare_data()

In [10]:
train_loader = build_loaders(train_df, tokenizer, mode="train")
valid_loader = build_loaders(valid_df, tokenizer, mode="valid")


Number of samples: 127135
Number of samples: 31780


In [18]:
import pandas as pd
df = pd.read_csv(f"{config.captions_path}/captions.csv")


In [19]:
print(len(df["image"].values),
      len(df["caption"].values))

In [17]:
train_loader.dataset[10000]

error: OpenCV(4.7.0) D:\bld\libopencv_1675730129252\work\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [9]:
train_df, valid_df = make_train_valid_dfs()
tokenizer = DistilBertTokenizer.from_pretrained(config.text_tokenizer)
train_loader = build_loaders(train_df, tokenizer, mode="train")
valid_loader = build_loaders(valid_df, tokenizer, mode="valid")


model = CLIPModel().to(config.device)
optimizer = torch.optim.AdamW(
    model.parameters(), lr=config.lr, weight_decay=config.weight_decay
)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", patience=config.patience, factor=config.factor
)
step = "epoch"

KeyboardInterrupt: 

In [8]:
best_loss = float('inf')
for epoch in range(config.epochs):
    print(f"Epoch: {epoch + 1}")
    model.train()
    train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(model, valid_loader)
    
    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg
        torch.save(model.state_dict(), "best.pt")
        print("Saved Best Model!")

Epoch: 1


  0%|          | 0/3973 [00:16<?, ?it/s]

error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "d:\anaconda3\envs\pytorch\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\envs\pytorch\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\envs\pytorch\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "d:\ACM AI\CLIP-Starter\filled_clip\utils\dataset.py", line 80, in __getitem__
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.7.0) D:\bld\libopencv_1675730129252\work\modules\imgproc\src\color.cpp:182: err